In [25]:
from functools import reduce
import os
import pandas as pd

The Dataverse team is planning to add  additions to the dataset deposit form in the Harvard Dataverse Repository, we need to know:

- In Dataverse repositories, how many datasets have funding metadata?
- And among those, how many have funder names?
- Which users include funding metadata in their deposits most often?

## Prepare the data

In [29]:
os.chdir('..')
print(os.getcwd())

/


In [27]:
# Import CSV file that lists PIDs of all datasets and which Dataverse installation
# they're published in, removing the PIDs of datasets whose metadata could not be
# saved, i.e. dataverse_json_export_saved is FALSE
datasetPIDsDF = (pd
    .read_csv(
        './metadata/dataset_pids_from_most_known_dataverse_installations.csv',
        usecols=lambda x: x not in ['dataset_pid', 'dataverse_name'],
        sep=',', na_filter=False)
    .query('(dataverse_json_export_saved == True)')
    .drop(columns=['dataverse_json_export_saved'])
    .reset_index(drop=True, inplace=False)
)

datasetPIDsDF.head()

FileNotFoundError: [Errno 2] No such file or directory: './metadata/dataset_pids_from_most_known_dataverse_installations.csv'

In [ ]:
# Import basic metadata of all dataset versions
datasetVersionMetadataDF = pd.read_csv(
    './metadata/basic_metadata_2022.10.02-2022.10.03.csv',
    usecols=lambda x: x not in [
        'dataset_pid', 'dataset_publication_date',
        'dataset_version_state', 'publisher'],
    parse_dates=['dataset_version_create_time'],
    sep=',', na_filter=False)

datasetVersionMetadataDF.head()

In [ ]:
print(f'Count of datasets: {len(datasetPIDsDF)}')
print(f'Count of dataset versions: {len(datasetVersionMetadataDF)}')

In [ ]:
# From the datasetVersionMetadataDF, lets use the version publication dates to get the PIDs and dataset version numbers of only the most recently published versions of each dataset. The resulting dataframe should contain the same number of rows as the datasetPIDsDF.

latestDatasetVersionsDF = (datasetVersionMetadataDF
    .iloc[
        datasetVersionMetadataDF
        .groupby('dataset_pid_url')['dataset_version_create_time']
        .agg(pd.Series.idxmax)]
    .reset_index(drop=True, inplace=False))

latestDatasetVersionsDF.head()

In [ ]:
print(f'Count of datasets: {len(datasetPIDsDF)}')
print(f'Count of rows in latestDatasetVersionsDF: {len(latestDatasetVersionsDF)}')

In [ ]:
# Join the latestDatasetVersionsDF and the datasetPIDsDF to add the installation column,
# so we know which installations published each dataset

basicDatasetMetadataDF = (pd
    .merge(latestDatasetVersionsDF, datasetPIDsDF,
        how='inner',
        on=['dataset_pid_url'])
    .reset_index(drop=True, inplace=False))

# Make sure the count of rows is the same as the count of total datasets: 340,857
print(len(basicDatasetMetadataDF))

In [ ]:
basicDatasetMetadataDF.head()

In [ ]:
# basicDatasetMetadataDF.to_csv('basicDatasetMetadataDF.csv', index=False)

In [ ]:
# Import funding metadata, dropping the dataset_pid column
grantInformationMetadataDF = pd.read_csv(
    './metadata/grant_information(citation)_2022.10.02-2022.10.03.csv',
    usecols=lambda x: x not in ['dataset_pid'],
    # sep=',', na_filter=False)
    sep=',')

print(f'Count of rows in grantInformationMetadataDF: {len(grantInformationMetadataDF)}')

In [ ]:
# Join grantInformationMetadataDF with basicDatasetMetadataDF to retain metadata of
# each dataset's latest version

grantInformationLatestVersionDF = (pd
    .merge(grantInformationMetadataDF, basicDatasetMetadataDF,
        how='inner',
        on=['dataset_pid_url', 'dataset_version_number'])
    .drop(columns=[
        'dataset_version_create_time', 'installation',
        'dataverse_alias'])
    .reset_index(drop=True, inplace=False))

In [ ]:
# Import contributor metadata, where a funding agency might be listed as a contributor
contributorMetadataDF = pd.read_csv(
    './metadata/contributor(citation)_2022.10.02-2022.10.03.csv',
    usecols=lambda x: x not in ['dataset_pid'],
    sep=',', na_filter=False)

print(len(contributorMetadataDF))

In [ ]:
# Join contributorMetadataDF with basicDatasetMetadataDF to retain metadata of
# each dataset's latest version

contributorLatestVersionDF = (pd
    .merge(contributorMetadataDF, basicDatasetMetadataDF,
        how='inner',
        on=['dataset_pid_url', 'dataset_version_number'])
    .drop(columns=[
        'dataset_version_create_time', 'installation',
        'dataverse_alias'])
    .reset_index(drop=True, inplace=False))

# contributorLatestVersionDF = contributorLatestVersionDF.drop(columns=[
#     'dataset_version_create_time', 'installation', 'dataverse_alias'])

In [ ]:
# Check the columns on all three dataframes basicDatasetMetadataDF, grantInformationLatestVersionDF and contributorLatestVersionDF
# All three dataframes should have the 'dataset_pid_url' and 'dataset_version_number' columns

print('Columns in basicDatasetMetadataDF:')
for i in list(basicDatasetMetadataDF.columns):
    print(i)
print(f'\nColumns in grantInformationLatestVersionDF:')
for i in list(grantInformationLatestVersionDF.columns):
    print(i)
print(f'\nColumns in contributorLatestVersionDF:')
for i in list(contributorLatestVersionDF.columns):
    print(i)

In [ ]:
# Combine the basicDatasetMetadataDF, grantInformationLatestVersionDF, and contributorLatestVersionDF with a
# full outer join on dataset_pid_url and dataset_version_number columns

dataframes = [basicDatasetMetadataDF, grantInformationLatestVersionDF, contributorLatestVersionDF]
indexList = ['dataset_pid_url', 'dataset_version_number']
for df in dataframes:
    df.set_index(indexList, inplace=True)

In [ ]:

fundingDatasetMetadataInDataverseInstallationsDF = reduce(lambda left, right: left.join(right, how='outer'), dataframes)
fundingDatasetMetadataInDataverseInstallationsDF = fundingDatasetMetadataInDataverseInstallationsDF.reset_index(drop=False, inplace=False)
fundingDatasetMetadataInDataverseInstallationsDF.head()

In [ ]:
# Finally, make sure that the number of datasets in the dataframe match the number of total datasets: 340,857
countOfDatasetsInfundingDatasetMetadataInDataverseInstallationsDF = len(pd.unique(fundingDatasetMetadataInDataverseInstallationsDF['dataset_pid_url']))
print(f'Number of datasets in fundingDatasetMetadataInDataverseInstallationsDF: {countOfDatasetsInfundingDatasetMetadataInDataverseInstallationsDF}')

In [ ]:
fundingDatasetMetadataInDataverseInstallationsDF

In [ ]:
# fundingDatasetMetadataInDataverseInstallationsDF.to_csv('fundingDatasetMetadataInDataverseInstallationsDF.csv', index=False)

## Exploring the data

Now that we've got the funding metadata of the latest versions of all datasets in the Dataverse installations, let's start answering our questions

### Collections with most funding metadata and most used funder agency names
- In the Harvard Dataverse Repository, which collections have datasets with the most funding metadata? This will help us figure out who to learn from when we make changes to how funding metadata is entered.
- Which funder agency names are entered most often? Knowing that might help us figure out how effective our efforts to standardize funder agency name metadata could be? For example, do the changes to the metadata fields (the "CV javascript") make it easier for depositors to enter the most popular funder agency names? How much easier?

In [ ]:
# Lets start by creating a dataframe containing only metadata of datasets published in the Harvard Dataverse Repository
datasetInHDVDF = (fundingDatasetMetadataInDataverseInstallationsDF
    .query('(installation == "Harvard Dataverse")')
    .drop(columns=['installation'])
    .reset_index(drop=True, inplace=False)
    )

datasetInHDVDF.head()

In [ ]:
countOfDatasetsInHDVDF = len(pd.unique(datasetInHDVDF['dataset_pid_url']))
print(f'Number of datasets in datasetInHDVDF: {countOfDatasetsInHDVDF}')

In [ ]:
# Now let's keep the metadata of the datasets that have funding metadata in the three fields:
# grantNumberAgency, grantNumberValue, and contributorName when contributorType is "Funder"
fundingDatasetMetadataInHDVDF = (
    datasetInHDVDF.query(
        '(grantNumberAgency == grantNumberAgency) or\
        (grantNumberValue == grantNumberValue) or\
        (contributorType == "Funder" and contributorName == contributorName)')
     .reset_index(drop=True, inplace=False)
     )

In [ ]:
fundingDatasetMetadataInHDVDF.head()

In [ ]:
print(f'Number of datasets in fundingDatasetMetadataInHDVDF: {(len(pd.unique(fundingDatasetMetadataInHDVDF["dataset_pid_url"])))}')

So of the 80,278 datasets in the Harvard Dataverse Repository as of early October 2022, 30,222 had metadata about funding in one of the three fields where we expect it.

What's entered most often in the grantNumberAgency (Funding Information Name) field?

In [ ]:
# Create a new dataframe that lists each funder name entered in grantNumberAgency and the number of datasets with that funder name
datasetCountByFundingAgencyNamesInHDV = (
    fundingDatasetMetadataInHDVDF[['dataset_pid_url', 'grantNumberAgency']]
        .query('grantNumberAgency == grantNumberAgency')
        .drop_duplicates()
        .groupby(['grantNumberAgency']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False)
)

datasetCountByFundingAgencyNamesInHDV.head(10)

It's most likely that the Leon Levy collection has the most datasets with a funder name in their grantNumberAgency (Funding Information Name) field.

It also looks like, for datasets with other funder name metadata, we might have to deal with the variations in spellings, maybe using a deduplication or fuzzy matching library, in order to see which funders are added to the funding metadata most often. For example, this shows only 57 datasets with the value "NIH" in the Funding Information Agency field, but I've found more than 57 datasets created from NIH-funded research, with other values in the field, such as "National Institute of Health".

For now, let's see which collections have the most datasets with funding metadata, excluding the Leon Levy collections.

In [ ]:
datasetsInHDVByCollection_NoLevy = (
    fundingDatasetMetadataInHDVDF[['dataset_pid_url', 'dataverse_alias']]
    .drop_duplicates()
    .groupby(['dataverse_alias']).count()
    .query('~dataverse_alias.str.contains("levy_photos").values')
    .rename(columns={'dataset_pid_url': 'count_of_datasets'})
    .sort_values(by=['count_of_datasets'], ascending=False)
    .reset_index(drop=False, inplace=False)
)

datasetsInHDVByCollection_NoLevy.head(10)

The IFPRI collections (IFPRI and AfricaRISING), worldfish collection and CIAT collection have the most datasets with funding metadata. ("harvard" is the alias of repository's main collection, where anyone can add data.)

What have the depositors of those datasets entered in the metadata?


In [ ]:
# fundingDatasetMetadataInHDVDF.to_csv('fundingDatasetMetadataInHDVDF.csv', index=False)

In [ ]:
def funder_names_in_collections(dataframe, collectionAliasesList):
    funderNamesInCollectionDF = (
        # Drop all but the needed columns
        dataframe[[
            'dataset_pid_url', 'grantNumberAgency', 'dataverse_alias']]
        # Remove all datasets except those in given list of collection aliases
        .query(
            'dataverse_alias in @collectionAliasesList')
        # Remove the 'dataverse_alias' column
        .drop(columns=['dataverse_alias'])
        # Keep only datasets that have funding agency name metadata
        .query('grantNumberAgency == grantNumberAgency')
        # Drop duplicate rows
        .drop_duplicates()
        # Group by funding agency name
        .groupby(['grantNumberAgency']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        # Sort by count of occurrences of each funder name
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False))
    return funderNamesInCollectionDF


In [ ]:
datasetCountByFundingAgencyNamesIFPRI = funder_names_in_collections(
    dataframe=fundingDatasetMetadataInHDVDF,
    collectionAliasesList=['IFPRI', 'AfricaRISING'])

datasetCountByFundingAgencyNamesIFPRI.head()

In [ ]:
datasetCountByFundingAgencyNamesWorldfish = funder_names_in_collections(
    dataframe=fundingDatasetMetadataInHDVDF,
    collectionAliasesList=['worldfish'])

datasetCountByFundingAgencyNamesWorldfish.head()

# funderNamesList_Worldfish = list(set(datasetCountByFundingAgencyNamesWorldfish["grantNumberAgency"].values.tolist()))

In [ ]:
datasetCountByFundingAgencyNamesCIAT = funder_names_in_collections(
    dataframe=fundingDatasetMetadataInHDVDF,
    collectionAliasesList=[ # Aliases of collections I think are associated with CIAT
        'CIAT',
        'AgBio',
        'AICCRA',
        'CIFOR',
        'gender',
        'crp6',
        'dapa',
        'AllianceBioversityCIATFoodConsumer',
        'AllianceBioversityCIATLandscapes',
        'AllianceBioversityCIATClimate',
        'AllianceBioversityCIATBiodiversity',
        'AllianceBioversityCIATDigital',
        'AllianceBioversityCIATCrops4NH',
        'soils',
        'AllianceBioversityCIAT'
    ])

datasetCountByFundingAgencyNamesCIAT.head(20)

In [ ]:
datasetCountByFundingAgencyNamesDFEEP = funder_names_in_collections(
    dataframe=fundingDatasetMetadataInHDVDF,
    collectionAliasesList=['DFEEP', 'ipa', 'jpal'])

datasetCountByFundingAgencyNamesDFEEP.head()

### Duplicate funding metadata in Harvard Dataverse

There are two fields where depositors can enter the names of funders: In the Funding Information Agency field and in the Contributor field when they choose the Contributor Type "Funder". How often has this happened and who's done it? By learning these things, we can see how big the problem is and who's used both fields? And later we can ask those people why. We need to learn if the design of the fields are meeting some need that we weren't aware of.

For now, let's continue looking only at the latest version of each dataset. This might cause a further under count of the number of times this issue has actually occurred. For example, first version of a dataset might have both fields filled but the latest might have only one. By considering only the latest version of each dataset, we'll miss cases like this. Eventually we'll have to think about what to do about the metadata of previous dataset versions.

Questions
- In the Harvard Dataverse Repository, how many datasets have values in the Funder Information fields and in the Contributor field when Contributor Type is "Funder"?
- How many datasets have funding metadata in their Contributor field and not in their Funder Information fields?
- How often are the same values in both fields? For example, one dataset might have "NIH" in the Funder Information Agency field and in the Contributor Name field when the Contributor Type is "Funder". How often do things like this happen?
- How often are different values in both fields?

In [ ]:
# Let's remind ourselves what information we have in the fundingDatasetMetadataInHDVDF dataframe
fundingDatasetMetadataInHDVDF.head()

In [ ]:
# Now let's query it to get only datasets that have values in both metadata fields
duplicateFundingFieldsInHDV = (fundingDatasetMetadataInHDVDF
        .query(
            'grantNumberAgency == grantNumberAgency and\
             (contributorType == "Funder" and contributorName == contributorName)')
        .sort_values(by=['dataset_pid_url'], ascending=True)
        .reset_index(drop=True, inplace=False))

duplicateFundingFieldsInHDV.head()

In [ ]:
datasetCountDuplicateFundingFields = len(pd.unique(duplicateFundingFieldsInHDV['dataset_pid_url']))
print(f'Number of datasets with metadata in both funding metadata fields: {datasetCountDuplicateFundingFields}')

In [ ]:
# duplicateFundingFieldsInHDV.to_csv('duplicateFundingFieldsInHDV.csv', index=False)

In [ ]:
# Let's see which collections have most of these datasets
countOfDuplicateFundingFieldsInHDVByCollection = (
    # Drop all but the needed columns
    duplicateFundingFieldsInHDV[[
        'dataset_pid_url', 'dataverse_alias']]
        # Drop duplicate rows
        .drop_duplicates()
        # Group by dataverse_alias
        .groupby(['dataverse_alias']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        # Sort by count of occurrences of each funder name
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False))

countOfDuplicateFundingFieldsInHDVByCollection.head(100)

How many datasets have funding metadata in their Contributor field and not in their Grant Information fields?

In [ ]:
contributorsButNoFundingInfomration = (fundingDatasetMetadataInHDVDF
    .query(
        '(contributorType == "Funder" and contributorName == contributorName)and\
        grantNumberAgency != grantNumberAgency')
    .reset_index(drop=True, inplace=False))

In [ ]:
contributorsButNoFundingInfomration.head()

In [ ]:
# contributorsButNoFundingInfomration.to_csv('contributorsButNoFundingInfomration.csv', index=False)

### Duplicate funder metadata in other Dataverse installations

In [ ]:
fundingDatasetMetadataInDataverseInstallationsDF.head()

In [ ]:
# fundingDatasetMetadataInDataverseInstallationsDF.to_csv('fundingDatasetMetadataInDataverseInstallationsDF.csv', index=False)

In [ ]:
countOfDatasetsInEachInstallationDF = (
    # Drop all but the needed columns
    fundingDatasetMetadataInDataverseInstallationsDF[[
        'dataset_pid_url', 'installation']]
        # Drop duplicate rows
        .drop_duplicates()
        # Group by dataverse_alias
        .groupby(['installation']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        # Sort by count of occurrences of each funder name
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False))

countOfDatasetsInEachInstallationDF.head()

In [ ]:
# Lets see how many of these datasets in each installation have values in both fields
duplicateFundingFieldsInAllInstallations = (fundingDatasetMetadataInDataverseInstallationsDF
    .query(
        'grantNumberAgency == grantNumberAgency and\
         (contributorType == "Funder" and contributorName == contributorName)')
    .sort_values(by=['dataset_pid_url'], ascending=True)
    .reset_index(drop=True, inplace=False))

duplicateFundingFieldsInAllInstallations.head()

In [ ]:
# duplicateFundingFieldsInAllInstallations.to_csv('duplicateFundingFieldsInAllInstallations.csv', index=False)

In [24]:
countOfDatasetsWithDuplicateFundingFieldsInEachInstallationDF = (
    # Drop all but the needed columns
    duplicateFundingFieldsInAllInstallations[[
        'dataset_pid_url', 'installation']]
        # Drop duplicate rows
        .drop_duplicates()
        # Group by dataverse_alias
        .groupby(['installation']).count()
        .rename(columns={'dataset_pid_url': 'count_of_datasets'})
        # Sort by count of occurrences of each funder name
        .sort_values(by=['count_of_datasets'], ascending=False)
        .reset_index(drop=False, inplace=False))

countOfDatasetsWithDuplicateFundingFieldsInEachInstallationDF.head(100)

NameError: name 'duplicateFundingFieldsInAllInstallations' is not defined